# Perfomance Analysis

In [5]:
import pandas as pd

## Sequential Model

In [ ]:
LOG_FILE = '../log/seq.csv'

df = pd.read_csv(LOG_FILE)
df['phase'] = df.replace({'phase': {0: 'train', 1: 'val'}})['phase']
df.head(n=10)

In [ ]:
df[df['phase'] == 'train']['batch_time(s)'].describe()

In [ ]:
df[df['phase'] == 'val']['batch_time(s)'].describe()

In [ ]:
df.groupby('epoch')['batch_time(s)'].sum().describe()

## Pipeline Model

It is necessary to study the computational time and space of the different types of layers before splitting the model. Considering that the studied model is composed of GATConv, Linear layer and Global Pool (for the graph embedding), the studies will be carried out on these three types of layers

In [ ]:
LAYER_LOG_FILE = '../log/layer.csv'

layer_df = pd.read_csv(LAYER_LOG_FILE)